# COMP 631 HW1

In [42]:
!pip install nltk
!pip install numpy
!pip install pandas

In [43]:
#import the nltk package
import nltk
#call the nltk downloader
nltk.download('all')

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Package abc is already up-to-date!
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Package alpino is already up-to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger is already up-
[nltk_data]    |       to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger_ru is already
[nltk_data]    |       up-to-date!
[nltk_data]    | Downloading package basque_grammars to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package basque_grammars is already up-to-date!
[nltk_data]    | Downloading package biocreative_ppi to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Pac

True

In [44]:
s1 = "Today DeepMind released AlphaCode!!!!!! Go AlphaCode!"
s2 = "The AlphaCode is released today."
s3 = "the AlphaCode won last week."
s4 = "Find the AlphaCode news last week."
s5 = "AlphaCode is a system released by DeepMind."
docs = [s1,s2,s3,s4,s5]

In [45]:
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer

# The Porter stemming algorithm
# is a process for removing the commoner morphological and 
# inflexional endings from words in English. I use it
# as part of a term normalisation process
lemmatizer = WordNetLemmatizer()

processed_text = []

# Pre-Processing

# Removing punctuations in string Using regex
def remove_punctuation(doc):
    doc = re.sub(r'[^\w\s]', '', doc)
    return doc

# Stop word removal and lemmoniztion
def remove_stop_words(doc):
    stop_words = set(stopwords.words('english'))
    new_text = ""
    words = word_tokenize(doc)
    for word in words:
        if word not in stop_words:
            word = lemmatizer.lemmatize(word)
            new_text = new_text + " " + word

    return new_text

def convert_lower_case(doc):
    doc = doc.casefold()
    return doc

def stemming(doc):
    porter = PorterStemmer()
    return porter.stem(doc)
    


def preprocess(data):
    data = convert_lower_case(data) # Case Folding
    data = remove_punctuation(data)
    data = remove_stop_words(data)
    data = stemming(data)
    return word_tokenize(data)

for s in docs:
    processed_text.append(preprocess(s))
    # print(preprocess(s))

processed_text

[['today', 'deepmind', 'released', 'alphacode', 'go', 'alphacod'],
 ['alphacode', 'released', 'today'],
 ['alphacode', 'last', 'week'],
 ['find', 'alphacode', 'news', 'last', 'week'],
 ['alphacode', 'system', 'released', 'deepmind']]

In [46]:

#2)  Construct the term-document incidence matrix based on the result of
# the question above. Report the resultant incidence matrix.

DF = {}
for i in range(len(processed_text)):
    tokens = processed_text[i]
    for w in tokens:
        try:
            DF[w].add(i)
        except:
            DF[w] = {i}

In [47]:
from collections import Counter
import numpy as np

def doc_freq(word):
    c = 0
    try:
        c = len(DF[word])
    except:
        pass
    return c

tfdict = {}
N = 5
doc = 0

for i in range(N):
    tokens = processed_text[i]
    counter = Counter(tokens)
    words_count = len(tokens)
    
    for token in np.unique(tokens):
        tf = counter[token]
        tfdict[doc, token] = tf

    doc += 1

tfdict

import pandas as pd
tfdict

total_vocab_size = len(DF)
total_vocab = [x for x in DF]

documents= [0, 1, 2, 3, 4]
docnames= ["doc 1", "doc 2","doc 3", "doc 4", "doc 5"]
final_df = pd.DataFrame(np.asarray([[(tfdict[(x,y)] if (x,y) in tfdict else 0) for x in documents] for y in total_vocab ]))
final_df.columns =  docnames
final_df.index =  total_vocab

final_df


,doc 1,doc 2,doc 3,doc 4,doc 5
today,1,1,0,0,0
deepmind,1,0,0,0,1
released,1,1,0,0,1
alphacode,1,1,1,1,1
go,1,0,0,0,0
alphacod,1,0,0,0,0
last,0,0,1,1,0
week,0,0,1,1,0
find,0,0,0,1,0
news,0,0,0,1,0


In [48]:
# 3) Implement TF-IDF on the term-document incidence matrix. Output the
# resultant TF-IDF matrix
from collections import Counter
import numpy as np

def doc_freq(word):
    c = 0
    try:
        c = len(DF[word])
    except:
        pass
    return c

tf_idf = {}
N = 5
doc = 0

for i in range(N):
    tokens = processed_text[i]
    counter = Counter(tokens)
    words_count = len(tokens)
    
    for token in np.unique(tokens):
        tf = counter[token]/words_count
        df = doc_freq(token)

        # if token == "alphacode":
        #     print(df)

        idf = np.log((N)/(df))
        tf_idf[doc, token] = tf*idf

    doc += 1
tf_idf

{(0, 'alphacod'): 0.26823965207235,
 (0, 'alphacode'): 0.0,
 (0, 'deepmind'): 0.15271512197902584,
 (0, 'go'): 0.26823965207235,
 (0, 'released'): 0.08513760396099845,
 (0, 'today'): 0.15271512197902584,
 (1, 'alphacode'): 0.0,
 (1, 'released'): 0.1702752079219969,
 (1, 'today'): 0.3054302439580517,
 (2, 'alphacode'): 0.0,
 (2, 'last'): 0.3054302439580517,
 (2, 'week'): 0.3054302439580517,
 (3, 'alphacode'): 0.0,
 (3, 'find'): 0.3218875824868201,
 (3, 'last'): 0.18325814637483104,
 (3, 'news'): 0.3218875824868201,
 (3, 'week'): 0.18325814637483104,
 (4, 'alphacode'): 0.0,
 (4, 'deepmind'): 0.22907268296853878,
 (4, 'released'): 0.12770640594149768,
 (4, 'system'): 0.40235947810852507}

In [49]:
import pandas as pd
tf_idf

total_vocab_size = len(DF)
total_vocab = [x for x in DF]

documents= [0, 1, 2, 3, 4]
docnames= ["doc 1", "doc 2","doc 3", "doc 4", "doc 5"]
final_df = pd.DataFrame(np.asarray([[(tf_idf[(x,y)] if (x,y) in tf_idf else 0) for x in documents] for y in total_vocab ]))
final_df.columns =  docnames
final_df.index =  total_vocab

final_df

,doc 1,doc 2,doc 3,doc 4,doc 5
today,0.152715,0.305430,0.00000,0.000000,0.000000
deepmind,0.152715,0.000000,0.00000,0.000000,0.229073
released,0.085138,0.170275,0.00000,0.000000,0.127706
alphacode,0.000000,0.000000,0.00000,0.000000,0.000000
go,0.268240,0.000000,0.00000,0.000000,0.000000
alphacod,0.268240,0.000000,0.00000,0.000000,0.000000
last,0.000000,0.000000,0.30543,0.183258,0.000000
week,0.000000,0.000000,0.30543,0.183258,0.000000
find,0.000000,0.000000,0.00000,0.321888,0.000000
news,0.000000,0.000000,0.00000,0.321888,0.000000


4) Measure the similarity between every pair of documents using cosine
similarity. Organize the output in a 5 ×5 matrix.

In [50]:
import math
import numpy as np 

# Define a cosine similarity function
# compute cosine similarity of v1 to v2: (a dot b)/{||a||*||b||)
def cosine_similarity(a,b):
    sumxx, sumxy, sumyy = 0, 0, 0
    for i in range(len(a)):
        x = a[i]; y = b[i]
        sumxx += x*x
        sumyy += y*y
        sumxy += x*y
    return sumxy/math.sqrt(sumxx*sumyy)

df2 = final_df
# Save the similarity index between the documents
def pair(s):
    for i, v1 in enumerate(s):
        for j in range(i+1, len(s)):
            yield [v1, s[j]]

dic={} 
documents= [0, 1, 2, 3, 4]
for (a,b) in list(pair(docnames)):
  dic[(a,b)] = cosine_similarity(df2[a].tolist(), df2[b].tolist())

# Print the cosine similarity index
print(dic)

{('doc 1', 'doc 2'): 0.3931333325941069, ('doc 1', 'doc 3'): 0.0, ('doc 1', 'doc 4'): 0.0, ('doc 1', 'doc 5'): 0.21467405799786557, ('doc 2', 'doc 3'): 0.0, ('doc 2', 'doc 4'): 0.0, ('doc 2', 'doc 5'): 0.12947399812101723, ('doc 3', 'doc 4'): 0.4947592105690922, ('doc 3', 'doc 5'): 0.0, ('doc 4', 'doc 5'): 0.0}


In [51]:
docnames= ["doc 1", "doc 2","doc 3", "doc 4", "doc 5"]

final_df = pd.DataFrame(np.asarray([[(dic[(x,y)] if (x,y) in dic else 0) for y in docnames] for x in docnames]))
final_df.columns =  docnames
final_df.index = docnames


final_df

,doc 1,doc 2,doc 3,doc 4,doc 5
doc 1,0.0,0.393133,0.0,0.000000,0.214674
doc 2,0.0,0.000000,0.0,0.000000,0.129474
doc 3,0.0,0.000000,0.0,0.494759,0.000000
doc 4,0.0,0.000000,0.0,0.000000,0.000000
doc 5,0.0,0.000000,0.0,0.000000,0.000000
